In [1]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>


<img src="http://www.nserc-crsng.gc.ca/_gui/wmms.gif" alt="Canada logo" align="right">

<br>

<img src="http://www.triumf.ca/sites/default/files/styles/gallery_large/public/images/nserc_crsng.gif?itok=H7AhTN_F" alt="NSERC logo" align="right" width = 90>



# Exploring NSERC Awards Data


Canada's [Open Government Portal](http://open.canada.ca/en) includes [NSERC Awards Data](http://open.canada.ca/data/en/dataset/c1b0f627-8c29-427c-ab73-33968ad9176e) from 1995 through 2016.

The awards data (in .csv format) were copied to an [Amazon Web Services S3 bucket](http://docs.aws.amazon.com/AmazonS3/latest/dev/UsingBucket.html).

> **Acknowledgement:** This notebook was constructed using JupyterHub service from [syzygy.ca](http://syzygy.ca) with assistance from [Ian Allison](https://github.com/ianabc) and India Heisz. -- [James Colliander](http://colliand.com)

## Load the data from AWS S3 into Jupyter

This process is loading NSERC Awards data starting with 1995 and moving to 2016...

In [2]:
## Thanks Ian Allison.
## Install some python libraries
import numpy as np
import pandas as pd
import sys

## Build a data frame with NSERC Awards Data by iteratively reading CSV files hosted on AWS S3.
## It would improve reproducibility if we imported from a definitive NSERC Awards Data.
df = pd.DataFrame()

startYear = 1995
endYear   = 2017  # The last year is not included, so 2017 means we include the 2016 collection but not 2017.

## some columns dropped here for convenience
for year in range(startYear, endYear):
    file = 'https://s3.ca-central-1.amazonaws.com/open-data-ro/NSERC/NSERC_GRT_FYR' + str(year) + '_AWARD.csv.gz'
    df = df.append(pd.read_csv(file, 
                               compression='gzip', 
                               usecols = [1, 2, 3, 4, 5, 7, 9, 11, 12, 13, 17, 28], 
                               encoding='latin-1'
                              )
                  )  
    print(year)
 

## Rename columns for better readability.
df.columns = ['Name', 'Department', 'OrganizationID',
                 'Institution', 'ProvinceEN', 'CountryEN',
                 'FiscalYear', 'AwardAmount', 'ProgramID',
                 'ProgramNameEN', 'Committee', 'ResearchSubjectEN']

## Strip out any leading or trailing whitespace in the ProgramID column
df['ProgramID'] = df['ProgramID'].str.strip();

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [3]:
import matplotlib.pyplot as plt

In [4]:
import plotly.tools as tls
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
print (__version__) # requires version >= 1.9.0
import cufflinks as cf

4.1.0


ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [ ]:
## run plotly offline
cf.go_offline()

In [ ]:
## via India Heisz
## Define some methods
## Plotly is not installed by default
## TODO Remove plotly dependence or arrange for plotly installed as default on the hub
import matplotlib.ticker as mtick
# import plotly.graph_objs as go
# import plotly.offline as py
#  from plotly.offline import init_notebook_mode, iplot
# import plotly.tools as tls
import matplotlib.pylab as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

def nsercPlot (data):
    fig, axes = plt.subplots()

    xAxis = 'FiscalYear'
    yAxis = 'AwardAmount'
    
    y = data.groupby(xAxis).sum()[yAxis]
    x = y.index
    
    plt.xlabel(xAxis, fontsize=14)
    plt.ylabel(yAxis, fontsize=14)
    plt.title(title)
      
    plt.plot(x,y)
    
    init_notebook_mode(connected=True)
    
    axes.scatter(x,y,s=plotPointSizes,alpha=opacity)
    canvas = FigureCanvas(fig)
    plotly_fig = tls.mpl_to_plotly(fig)
    py.iplot(plotly_fig)
    
    return;

def viewAvailableSearch(column, searchString):
    available = df.drop_duplicates(subset = column)
    available = available[available[column].str.contains(searchString, na=False)]
    sorted = available.sort_values(by=[column], ascending=[True])
    print(sorted.to_string(columns= [column], index=False))
    return;

def overview(column, data):
    mean = data[columnYouWantInformationOn].mean()
    print('The mean of ' + str(columnYouWantInformationOn) + ' is ' + str(mean))

    median = data[columnYouWantInformationOn].median()
    print('The median of ' + str(columnYouWantInformationOn) + ' is ' + str(median))

    standardDeviation = data[columnYouWantInformationOn].std()
    print('The standard deviation of ' + str(columnYouWantInformationOn) + ' is ' + str(standardDeviation))

    awardCount = data.AwardAmount.count()
    print('The total number of awards for your selection is ' + str(awardCount))
    return;

## First Look at the Data

An exploration of the table.

In [ ]:
df
## Display the dataframe on the screen as a table. 

### Number of rows in Awards Data Frame

In [ ]:
## Number of rows representing all award reports from NSERC during 1995-2016
len(df)

### Columns of Data Frame

In [ ]:
df.columns

Money flowed out of NSERC? We have a data set that reports NSERC disbursements with data to answer:
> When? Who received it? Which university? Province? How much? Which NSERC program?

## Values in Columns

In [ ]:
df['Department'].unique()

### Number of unique departments

In [ ]:
## Number of unique departments
len(df['Department'].unique())

In [ ]:
df['ProvinceEN'].unique()

In [ ]:
# How many unique ProgramID values?
len(df['ProgramID'].unique())

In [ ]:
## Mismatch in program labels?
## TODO: Match ProgramNameEN to ProgramID; Find outliers
len(df['ProgramNameEN'].unique())

### Programs by AwardAmount

In [ ]:
## Identify programs that have received the most funding
pd.pivot_table(df, index=['ProgramNameEN','FiscalYear'], aggfunc=sum,
               values = ['AwardAmount']
              ).sort_values(by=['AwardAmount'], ascending=False)

In [ ]:
## Identify programs that have received the most funding
pd.pivot_table(df, index=['ProgramNameEN'], aggfunc=sum,
               values = ['AwardAmount']
              ).sort_values(by=['AwardAmount'], ascending=False)

In [ ]:
## Identify programs that have received the most funding
pd.pivot_table(df, index=['FiscalYear', 'ProgramNameEN'], aggfunc=sum,
               values = ['AwardAmount']
              ).sort_values(by=['AwardAmount'], ascending=False)

In [ ]:
programYear = pd.pivot_table(df, index=['ProgramNameEN','FiscalYear'], aggfunc=sum,
               values = ['AwardAmount']
              ).sort_values(by=['AwardAmount'], ascending=False)

In [ ]:
## Export this table as a CSV file.
programYear.to_csv('program-year.csv')

## Big Winners

In [ ]:
## Historical Program recipents of most NSERC funds: 1995 -- 2016
df.groupby(['ProgramNameEN']).sum()['AwardAmount'].sort_values(ascending=False).head(n=10)

In [ ]:
## Historical PI recipents of most NSERC funds: 1995 -- 2016
df.groupby(['Name']).sum()['AwardAmount'].sort_values(ascending=False).head(n=50)

## Discovery and Not Discovery Budgets

### Total Budget

In [ ]:
ax = (df.groupby(['FiscalYear']).sum()['AwardAmount'].plot(kind="bar", title="Total Expenditures vs. Time"))
ax.set_ylabel("Reported Dollars")

In [ ]:
df.groupby(['FiscalYear']).sum()['AwardAmount'].iplot(kind="bar")

In [ ]:
## Total Annual Budget
df.groupby(['FiscalYear']).sum()['AwardAmount']

### Not Discovery

In [ ]:
## The ~ operator negates the condition.
ax = (df.loc[~(df['ProgramNameEN'].str.contains('iscovery'))].groupby(['FiscalYear']).sum()['AwardAmount'].plot(kind="bar", title="Not containing 'iscovery' vs. Time"))
ax.set_ylabel("Reported Dollars")

In [ ]:
df.loc[~(df['ProgramNameEN'].str.contains('iscovery'))].groupby(['FiscalYear']).sum()['AwardAmount'].iplot(
    kind="bar", title="Not containing 'iscovery' vs. Time")

In [ ]:
df.loc[~(df['ProgramNameEN'].str.contains('iscovery'))].groupby(['FiscalYear']).sum()['AwardAmount']

### Discovery

In [ ]:

ax = (df.loc[(df['ProgramNameEN'].str.contains('iscovery'))].groupby(['FiscalYear']).sum()['AwardAmount']
      .plot(kind="bar", title="Containing 'iscovery' vs. Time"))
ax.set_ylabel("Reported Dollars")

In [ ]:
df.loc[(df['ProgramNameEN'].str.contains('iscovery'))].groupby(
    ['FiscalYear']).sum()['AwardAmount'].iplot(kind="bar", title="Containing 'iscovery' vs. Time")

In [ ]:
df.loc[(df['ProgramNameEN'].str.contains('iscovery'))].groupby(['FiscalYear']).sum()['AwardAmount']

In [ ]:
disc = df.loc[(df['ProgramNameEN'].str.contains('iscovery'))]

In [ ]:
disc['ProgramNameEN'].unique()

In [ ]:
## Distribution of Discovery Funding Across Programs
(disc.groupby(['ProgramNameEN','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Total dollars invested by Evaluation Group Committee")

In [ ]:
(disc.groupby(['ProgramNameEN','FiscalYear']).sum()
 ['AwardAmount'].unstack()).iplot(
    kind="bar")

The bulk of the historical expenses in Discovyer took place in "Discovery Grants Program - Individual"

### Not Discovery

In [ ]:
notdisc = df.loc[~(df['ProgramNameEN'].str.contains('iscovery'))]

In [ ]:
notdisc['ProgramNameEN'].unique()

In [ ]:
## Distribution of Discovery Funding Across Programs
(notdisc.groupby(['ProgramNameEN','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=True, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Reported Dollars invested across programs outside of Discovery")

In [ ]:
notdisc.groupby(['ProgramNameEN','FiscalYear']).sum()['AwardAmount'].unstack().iplot(
    kind="bar", 
    title="Reported Dollars invested across programs outside of Discovery")

## Evaluation Group (EG) Committees

![](https://wwejubwfy.s3.amazonaws.com/NSERC_-_Selection_Committees_and_Evaluation_Groups_2018-04-12_14-37-32.png)

In [ ]:
committee = 1504
ax = df.loc[df['Committee'] == committee].groupby(['FiscalYear']).sum()['AwardAmount'].plot(
    kind="bar", title="committee vs. Time")
ax.set_ylabel("Reported Dollars")

In [ ]:
EvaluationGroups = df.loc[(df['Committee']==1501) 
                                   | (df['Committee']==1502)
                                   | (df['Committee']==1503)
                                   | (df['Committee']==1504) 
                                   | (df['Committee']==1505)
                                   | (df['Committee']==1506)
                                   | (df['Committee']==1507)
                                   | (df['Committee']==1508)
                                   | (df['Committee']==1509)
                                   | (df['Committee']==1510)
                                   | (df['Committee']==1511)
                                   | (df['Committee']==1512)]

In [ ]:
## 2009 is an outlier so drop it. 2010 is also a bit weird.
EG = EvaluationGroups.loc[~(EvaluationGroups['FiscalYear']==2009)]

In [ ]:
EG['FiscalYear'].unique()

## Total Reported Dollars vs Fiscal Year separated by EG Committee

In [ ]:
## Total investment dollars vs Fiscal Year separated by Evaluation Group Committee
(EG.groupby(['Committee','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Total dollars invested by Evaluation Group Committee")

In [ ]:
## Total investment dollars vs Fiscal Year separated by Evaluation Group Committee
(EG.groupby(['Committee','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Total dollars invested by Evaluation Group Committee")

Discipline Dynamics? The number and age of PIs served by each committee and each fiscal year changes.

## Number of Awards vs Fiscal Year separated by EG Committee

In [ ]:
## Number of Awards vs Fiscal Year separated by Evaluation Group Committee
(EG.groupby(['Committee','FiscalYear']).count()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Number of Awards by Evaluation Group Committee")

Calculuate an average? Divide the total dollars spent on a committee during a fiscal year 
by the number of awards given in that committee during that fiscal year.

## Average Award Size vs. Fiscal Year separated by EG Committee

In [ ]:
## Total Dollars invested divided by number of awards vs. Fiscal Year separated by Evaluation Group Committee
((EG.groupby(['Committee','FiscalYear']).sum()
 ['AwardAmount'])/(EvaluationGroups.groupby(['Committee','FiscalYear']).count()
 ['AwardAmount'])).unstack().plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Average Award Amount by Evaluation Group Committee")

### Calculate using .mean()

In [ ]:
## Mean vs Fiscal Year separated by Evaluation Group Committee
(EG.groupby(['Committee','FiscalYear']).mean()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="Number of Awards by Evaluation Group Committee")

In [ ]:
## (Stacked) Sum vs Fiscal Year separated by Evaluation Group Committee
(EG.groupby(['Committee','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=True, cmap="viridis", figsize=(16, 8), 
    title="Sum of Awards by Evaluation Group Committee")

## Explore Funding of Major Programs by Institution

## Canada Research Chairs

In [ ]:
crc = df.loc[df['ProgramNameEN']=='Canada Research Chairs']

In [ ]:
## CRCs by Instititution
(crc.groupby(['Institution','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=True, cmap="viridis", alpha=0.7, figsize=(24, 8), width=1, title="CRCs by Institution")

## CRD

In [ ]:
crd = df.loc[df['ProgramNameEN'] == 'Collaborative Research and Development Grants']

In [ ]:
## Number of Awards vs Fiscal Year separated by Evaluation Group Committee
(crd.groupby(['Institution','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=True, cmap="viridis", alpha=0.7, figsize=(24, 8), width=1, title = "CRDs by Institution")

## Strategic Projects

In [ ]:
strat = df.loc[df['ProgramNameEN'] == 'Strategic Projects - Group']

In [ ]:
## Strategic Project Awards vs Fiscal Year separated by Institution
(strat.groupby(['Institution','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=True, cmap="viridis", alpha=0.7, figsize=(24, 8), width=1, 
    title="Strategic Projects -- Group by Institution")

## Discovery Grants - Individual

In [ ]:
discov = df.loc[df['ProgramNameEN'] == 'Discovery Grants Program - Individual']

In [ ]:
## Strategic Project Awards vs Fiscal Year separated by Institution
(discov.groupby(['Institution','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=True, cmap="viridis", alpha=0.7, figsize=(24, 8), width=1,
    title="Discover Grants -- Individual by Institution")

### Postdocs

In [ ]:
viewAvailableSearch('ProgramNameEN', 'ostdo')

In [ ]:
pdf = df.loc[df['ProgramNameEN']=='Postdoctoral Fellowships']

In [ ]:
pdf

In [ ]:
(pdf.groupby(['ProgramID','FiscalYear']).sum()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="")

In [ ]:
(pdf.groupby(['ProgramNameEN','FiscalYear']).count()
 ['AwardAmount'].unstack()).plot(
    kind="bar", stacked=False, cmap="viridis", alpha=0.7, figsize=(16, 8), width=1, 
    title="")

## Percentage Growth Across Evaluation Groups

In [ ]:
(EG.groupby(['Committee','FiscalYear']).sum()
 ['AwardAmount'].unstack())

In [ ]:
awards = pd.read_csv("./eg-vs-time.csv")

In [ ]:
awards

In [ ]:
awards.columns

In [ ]:
aw = awards[['FiscalYear', 'EGTotal', '1501', '1502', '1503', '1504',
       '1505', '1506', '1507', '1508', '1509', '1510', '1511', '1512']]

In [ ]:
aw

In [ ]:
aw

In [ ]:
norm = aw.divide(aw.iloc[3])

In [ ]:
norm.plot(cmap="viridis")

In [ ]:
norm[['EGTotal','1508']].plot()

In [ ]:
norm[['EGTotal','1508']].plot()

In [ ]:
norm.plot()

In [ ]:
norm

In [ ]:
ctrms = df.loc[df['ProgramID']=='CTRMS']

In [ ]:
(ctrms.groupby('FiscalYear')['AwardAmount'].sum()/2656500).plot()

In [ ]:
(ctrms.groupby('FiscalYear')['AwardAmount'].sum()).plot()

In [ ]:
ctrms